# Boostrapers

## Yolov5 Classification and Bouding Box detection

### Classification success estimate >99%

In [19]:
!pip install opencv-python numpy matplotlib

import cv2
import numpy as np
from PIL import Image


import time
import sys
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches

Cleaning up...


In [20]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

path_annotrain = "./datasets_train/train_annotation/_annotation.csv"

train_annotation = pd.read_csv(path_annotrain, index_col=0)

bbox_train_full = train_annotation.loc[
    :, ["im_name", "x_min", "y_min", "x_max", "y_max", "class"]
]

bbox_train_full['isCar'] = (bbox_train_full['class'] == "car")

In [21]:
from sklearn.model_selection import train_test_split

bbox_train, bbox_eval = train_test_split(
    bbox_train_full, test_size=0.1, random_state=1234
)

In [22]:
print(bbox_train_full)

               im_name        x_min       y_min        x_max        y_max  \
0        107347968.jpg    40.000000  244.000000  1144.000000   637.000000   
1         91160576.jpg     4.000000    9.000000    98.000000    76.000000   
2        273809408.jpg    16.000000    4.000000   288.000000   191.000000   
3        579534848.jpg    14.000000   23.000000   255.000000   178.000000   
4        950599680.jpg     4.000000   70.000000   321.000000   215.000000   
...                ...          ...         ...          ...          ...   
9690    9403488242.jpg   132.238000  371.477774  1871.606000  1200.390494   
14225   2224571280.jpg   242.737881  131.401610   686.737005   508.379060   
15669    146443543.jpg   457.746240   69.630292  2103.768560  1532.364560   
16445  15666905933.jpg  1365.900768  875.976000  1537.965792  1005.971328   
9872    8379986555.jpg     0.000000    0.000000   798.000000  1200.000000   

              class  isCar  
0               car   True  
1               c

In [23]:
print(bbox_train)

               im_name       x_min        y_min        x_max        y_max  \
463      367919104.jpg   12.000000    23.000000   245.000000   176.000000   
1102      42991616.jpg   24.000000    87.000000   365.000000   255.000000   
21510  17379700870.jpg  413.602500   189.986310  1097.998500   796.654170   
62      1007747072.jpg   43.000000    48.000000   611.000000   306.000000   
774        4521984.jpg   24.000000   141.000000   721.000000   396.000000   
...                ...         ...          ...          ...          ...   
279      951386112.jpg   18.000000   128.000000   607.000000   398.000000   
16590  14249824423.jpg  153.435744  1183.058784   268.728416  1265.862816   
664      863567872.jpg    1.000000    15.000000   640.000000   319.000000   
3039   16161138999.jpg  650.160000   107.572426   858.833000   611.557626   
723      742522880.jpg   80.000000    82.000000   831.000000   572.000000   

        class  isCar  
463       car   True  
1102      car   True  
21510 

In [39]:
!pip install opencv-python numpy matplotlib

import cv2
import numpy as np
from PIL import Image

import sys
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches

CONFIDENCE = 0.3 


# loading the YOLO neural network
config_path = "yolov3.cfg"
weights_path = "yolov3.weights"
labels = open("coco.names").read().strip().split("\n") 
net = cv2.dnn.readNetFromDarknet(config_path, weights_path) 


def second_step(im_name): 

    path_name = 'datasets_train/train/'+ im_name 
    image = cv2.imread(path_name)
    file_name = os.path.basename(path_name) 
    filename, ext = file_name.split(".") 

    h, w = image.shape[:2]
    
    # create 4D blob
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    # get the output predictions
    ln = net.getLayerNames()
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
    layer_outputs=net.forward(ln)
    
    
    #iteration over the neural network 
    boxes, confidences, class_ids = [], [], []
    for output in layer_outputs:
        for detection in output: #loop over each of the object detections
            scores = detection[5:]
            class_id = np.argmax(scores) 
            confidence = scores[class_id] 
            
            if (confidence > CONFIDENCE): 
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
            
    if ((2 not in class_ids) and (5 not in class_ids) and (7 not in class_ids)):  #we consider both buses and trucks as cars
        return [False, [0,0,0,0]] #no car in the output predictions
    
    else:
        indices = []
        legal_indices = [2,5,7]
                
        for i in range(len(boxes)):  
            if (class_ids[i] in legal_indices):
                indices.append(i)

        max = boxes[indices[0]][2] + boxes[indices[0]][3] #selection of the largest rectangle
        max_id = indices[0]

        for i in indices:
            if boxes[i][2] + boxes[i][3] > max:
                max = boxes[i][2] + boxes[i][3]
                max_id = i
        
        x_min, y_min, w, h = boxes[max_id]
        x_min, y_min, x_max, y_max = x_min, y_min, x_min + w, y_min + h
            
        return [True, [x_min, y_min, x_max, y_max]]
        

In [25]:
second_step('1000800256.jpg')

[True, [40, -10, 475, 185]]

In [26]:
# iterating over rows using iterrows() function
#errors = 0

#for i, j in bbox_train.iterrows():
#    im_name = j['im_name']
#    result = second_step(im_name)
#    if(result[0] != j['isCar']):
#        errors += 1
    
#print(errors / len(bbox_train))

Premature end of JPEG file
Premature end of JPEG file


KeyboardInterrupt: 

In [47]:
directory = './car_data/predict'


df = pd.DataFrame(columns=["im_name", "x_min", "y_min", "x_max", "y_max", "e"])

i = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    result = second_step('./car_data/predict/' + filename)
    
    #print(result[0], result[1][0], result[1][1], result[1][2], result[1][3])
    
    
    df = df.append({
     "im_name": filename,
     "classType": result[0],
     "x_min": result[1][0],
     "y_min": result[1][1],
     "x_max": result[1][2],
     "y_max": result[1][3],
      }, ignore_index=True)

print(df)

              im_name x_min y_min x_max y_max    e  classType
0        85854523.jpg    26   643  1834  1300  NaN        1.0
1     16663988681.jpg     0     0     0     0  NaN        0.0
2     19090334369.jpg     0     0     0     0  NaN        0.0
3        71540198.jpg    67    72  1783   762  NaN        1.0
4        85427036.jpg   122   169  1049   656  NaN        1.0
...               ...   ...   ...   ...   ...  ...        ...
1019     89862599.jpg    10    75   961   521  NaN        1.0
1020   7055251597.jpg     0     0     0     0  NaN        0.0
1021   3848520911.jpg     0     0     0     0  NaN        0.0
1022     73537501.jpg    53    71   528   365  NaN        1.0
1023     98186923.jpg   145   119   767   512  NaN        1.0

[1024 rows x 7 columns]


In [28]:
!cd ./car_data/predict && rm -rf .ipynb_checkpoints

In [1]:
df.to_csv('intermediate_solution.csv', index = False, header=True)

NameError: name 'df' is not defined

## Resnet34 Car Classifier

In [ ]:
!pip install fastai==1.0.39

from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
bs = 64

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
data_dir='./car_data/train'

list = os.listdir(data_dir) 
number_files = len(list)
print(number_files)

In [ ]:
data = ImageDataBunch.from_folder(path,  
                                  valid_pct=0.2,
                                  ds_tfms=get_transforms(do_flip=True,flip_vert=False, max_rotate=90),
                                  size=224,bs=64, 
                                  num_workers=0).normalize(imagenet_stats)

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=accuracy, model_dir="/tmp/model/")

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze() 
learn.fit_one_cycle(4, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.export()

In [ ]:
#image = Image.open("../input/final-car-train/car_data/test/AM General Hummer SUV 2000/00076.jpg")
#x = pil2tensor(image, np.float32)
#preds_num = learn.predict(Image(x))[2].numpy()

#trn_tfms, val_tfms = tfms_from_model(arch,sz) # get transformations
im = open_image("../input/final-car-train/car_data/test/Audi 100 Wagon 1994/00192.jpg")
#learn.precompute=False # We'll pass in a raw image, not activations
#reds = learn.predict_array(im[None])
#p.argmax(preds) # preds are log probabilities of classes

pred_class, pred_idx, outputs =   learn.predict(im)